In [15]:
import sys
sys.path.append('../')
from data.dataset import *
import tensorflow as tf
import tensorflow.keras.layers as tfkl
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time

Cargo el dataset de texto y le aplico las transformaciones para poder usarlo para entrenar

In [12]:
path_to_file = '../data/txts/chamame_dataset.txt'
batch_size = 32
train_size = 90

encoded_data, max_id = dataset_pre(path_to_file)
data_train = batch_stateful(encoded_data,train_size,batch_size,100,max_id)

Defino el modelo, dejando las partes del dropout por defecto para que use la GPU (si tuviera una :( )

In [13]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

Defino la clase para resetear los estados del modelo al final de cada época

In [14]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

Entreno el modelo

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])